In [ ]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import WeightedRandomSampler,RandomSampler, DataLoader, random_split
import random
import numpy as np
import os

In [ ]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset = datasets.ImageFolder(root='Rice Leaf Disease Images',transform=transform)

In [ ]:
class_weight=[]
for root, subdir, files in os.walk('Rice Leaf Disease Images'):
    if len(files)>0:
        class_weight.append(1./len(files))
sample_weight = [ class_weight[e] for _,e in dataset.samples]


In [ ]:
sampler = WeightedRandomSampler(sample_weight,num_samples=len(sample_weight),replacement=True)

In [ ]:
total_size = len(dataset)
train_size = int(0.70 * total_size)
val_size =int(0.15 * total_size)
test_size = total_size-train_size-val_size

In [ ]:
train_dataset,val_dataset,test_dataset = random_split(dataset,[train_size,val_size,test_size])

In [ ]:
train_loader = DataLoader(train_dataset,batch_size=6)
test_loader = DataLoader(test_dataset,batch_size=6)
val_loader = DataLoader(val_dataset,batch_size=6)

In [ ]:
# model = models.resnet50(pretrained=True)
model = models.resnet18(pretrained=True)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
num_feature = model.fc.in_features

In [ ]:
num_classes = dataset.classes
model.fc = torch.nn.Linear(num_feature,len(num_classes))

In [ ]:
criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.006, momentum=0.875,weight_decay= 3.051757812)


In [ ]:
def accuracy_metric(outputs,targets):
    _,preds = torch.max(outputs,dim=1)
    correct = torch.sum(preds==targets)
    accuracy = correct.float()/targets.size(0)
    return accuracy

In [ ]:
device = torch.device("cudo:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [21]:
for epoch in range(250):
    model.train()
    running_loss =0.0
    running_accuracy=0.0
    for inputs,labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_accuracy+= accuracy_metric(outputs,labels).item()*inputs.size(0)
        epoch_loss = running_loss/ len(train_loader.dataset)
        epoch_accuracy = running_accuracy / len(train_loader.dataset)
    print(f'Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}, Accuracy:{epoch_accuracy:.4f}')

Epoch 1, Loss: 1.386128625773281, Accuracy:0.2526


In [20]:
torch.save(model,"ResNet50-Rice-Disease-Detection-v3.pt")